In [1]:
pip install bert-for-tf2

     |████████████████████████████████| 51kB 6.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=9b7e28ee71008a5a604fbd9437491862192f769a48cf1d5dda82b0ae61dd8eb3
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=84bb20c3e6b7ad5d25cf0ba5f76b4c0aeb9f0570de25b59280cee1a6243227a8
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=6b678fc73521bdd66e9af9791c724e3907e0db191b00433a1e884ec2f5ef9b12
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [2]:
pip install sentencepiece

     |████████████████████████████████| 1.2MB 12.0MB/s 


In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [4]:
import pandas as pd
import re
import numpy as np
import random
import math

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [6]:
data = pd.read_csv('/gdrive/My Drive/github/интерсвязь - тестовое задание/tatarstan_message_2.csv', sep = ';')
print(data.columns)
data = data[['categories', 'description_stem']]
data.head(3)

Index(['Unnamed: 0', 'categories', 'description', 'description_prep',
       'description_stem'],
      dtype='object')


,categories,description_stem
0,Благоустройство территории,трасса нижнекамск чистополь лесополоса пгт кам...
1,Благоустройство территории,двор дом ленинградский отсутствовать парковочн...
2,Содержание и ремонт муниципальных дорог,добрый день осуществляться уборка снег центр г...


In [7]:
X = [x for x in data['description_stem']]
Y = pd.get_dummies(data['categories'])
#Y = np.argmax(np.array(Y), axis = 1)
#Y =  data.categories.replace(data.categories.unique(), range(data.categories.unique().shape[0]), inplace=False) 
print(Y)
#print(data.categories.unique().shape[0])

       Cвалки  ...  Цифровое телевидение
0           0  ...                     0
1           0  ...                     0
2           0  ...                     0
3           0  ...                     0
4           0  ...                     0
...       ...  ...                   ...
29017       0  ...                     0
29018       0  ...                     0
29019       0  ...                     0
29020       0  ...                     0
29021       0  ...                     0

[29022 rows x 36 columns]


In [8]:
data.categories.unique()

array(['Благоустройство территории',
       'Содержание и ремонт муниципальных дорог',
       'Поликлиники и больницы', 'Бездомные животные', 'Парки и скверы',
       'Мобильная связь', 'Организация дорожного движения',
       'Незаконные азартные игры', 'Санитарное состояние',
       'Нарушение в наружной рекламе', 'Работа спортивных объектов',
       'Капитальный ремонт', 'Общественный транспорт', 'Воздух',
       'Садоводческие, огороднические и дачные некоммерческие объединения',
       'Детские сады', 'Доступная среда', 'Жилищное строительство',
       'Почта', 'Нарушение правил торговли', 'Пенсионное обеспечение',
       'Вода', nan,
       'Содержание и ремонт федеральных и республиканских дорог',
       'Меры социальной поддержки, предоставляемые органами социальной защиты',
       'Cвалки', 'Работа учреждений культуры', 'Пособия безработным',
       'Социальное обслуживание', 'Объекты культурного наследия',
       'Сельское хозяйство', 'Цифровое телевидение',
       'Тех.средс

In [9]:
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file="/gdrive/My Drive/github/интерсвязь - тестовое задание/bert_model/vocab.txt",
     do_lower_case=True)
bert_layer = hub.KerasLayer("/gdrive/My Drive/github/интерсвязь - тестовое задание/bert_model",
                            trainable=False, signature='tokens' , signature_outputs_as_dict=True)
#vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
#to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
#tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [10]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'j', '##ud', '##g', '##ment', '##al']

In [11]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[53106, 443, 11018, 27126, 263, 9037, 474, 12947, 1672]

In [12]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [13]:
tokenized_reviews = [tokenize_reviews(str(review)) for review in X]

In [14]:
print(tokenized_reviews[0])

[30914, 70528, 2059, 932, 5850, 4659, 107394, 754, 58791, 3511, 727, 378, 87244, 11243, 732, 378, 2134, 12409, 58087, 673, 73507, 816, 12737, 65511, 726, 1165, 27183, 87197, 13544, 58619]


In [15]:
from keras.preprocessing.sequence import pad_sequences
SENTENCE_LENGTH = 50

tokenized_reviews = pad_sequences(tokenized_reviews, maxlen=SENTENCE_LENGTH)
reviews_with_len = [[review, Y.iloc[i]]
                 for i, review in enumerate(tokenized_reviews)]

In [16]:
print(reviews_with_len[0])

[array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,  30914,  70528,   2059,    932,
         5850,   4659, 107394,    754,  58791,   3511,    727,    378,
        87244,  11243,    732,    378,   2134,  12409,  58087,    673,
        73507,    816,  12737,  65511,    726,   1165,  27183,  87197,
        13544,  58619], dtype=int32), Cвалки                                                                                  0
Бездомные животные                                                                      0
Благоустройство территории                                                              1
Вода                                                                                    0
Воздух                                                                                  0
Детские сады                                                                         

In [17]:
random.shuffle(reviews_with_len)

In [18]:
from sklearn.model_selection import train_test_split

X = np.array([part[0] for part in reviews_with_len])
Y = np.array([part[1] for part in reviews_with_len])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [19]:
# Метрики 
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [20]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=36,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1, activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes, name='out_layer',
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [21]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 36

DROPOUT_RATE = 0.2

NB_EPOCHS = 100

In [22]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [23]:
text_model.compile(loss="binary_crossentropy",
                    optimizer="adam",
                    metrics=['accuracy', precision, recall, f1])

In [27]:
from keras.callbacks import ModelCheckpoint
from keras import backend as K 
checkpoint = ModelCheckpoint("/gdrive/My Drive/github/интерсвязь - тестовое задание/CNN_Bert_classifier-{epoch:02d}-{val_f1:.2f}.hdf5", # Сохраняем модель с лучшим val_f1
                             monitor='val_f1', save_best_only=True, mode='max')
history = text_model.fit(X_train, Y_train, batch_size=32, epochs=200, validation_split=0.2, callbacks = [checkpoint])

Epoch 1/200


KeyboardInterrupt: ignored

In [28]:
plt.style.use('ggplot')


def plot_metrix(ax, x1, x2, title):
    ax.plot(range(1, len(x1) + 1), x1, label='train')
    ax.plot(range(1, len(x2) + 1), x2, label='val')
    ax.set_ylabel(title)
    ax.set_xlabel('Epoch')
    ax.legend()
    ax.margins(0)


def plot_history(history):
    fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(16, 9))
    ax1, ax2, ax3, ax4 = axes.ravel()

    plot_metrix(ax1, history.history['precision'], history.history['val_precision'], 'Precision')
    plot_metrix(ax2, history.history['recall'], history.history['val_recall'], 'Recall')
    plot_metrix(ax3, history.history['f1'], history.history['val_f1'], "$F_1$")
    plot_metrix(ax4, history.history['loss'], history.history['val_loss'], 'Loss')
    plot_metrix(ax4, history.history['accuracy'], history.history['val_accuracy'], 'Accuracy')

    plt.show()


plot_history(history)

NameError: ignored

In [30]:
text_model.load_weights('/gdrive/My Drive/github/интерсвязь - тестовое задание/weights/CNN_Bert_classifier-05-0.81.hdf5')

In [32]:
from sklearn.metrics import classification_report
print("Оценки тестового набора:")
predicted = np.round(text_model.predict(X_test))
print(classification_report(Y_test, predicted, digits=3))

Оценки тестового набора:
              precision    recall  f1-score   support

           0      0.981     0.750     0.850        68
           1      0.987     0.993     0.990       148
           2      0.931     0.915     0.923      2574
           3      1.000     0.400     0.571         5
           4      0.914     0.914     0.914        35
           5      0.889     0.889     0.889        27
           6      0.000     0.000     0.000         0
           7      0.938     0.750     0.833        20
           8      0.875     0.778     0.824         9
           9      0.923     0.889     0.906        27
          10      1.000     1.000     1.000         2
          11      1.000     0.927     0.962        41
          12      0.949     0.947     0.948       413
          13      0.000     0.000     0.000         0
          14      0.000     0.000     0.000         1
          15      1.000     0.667     0.800         3
          16      0.916     0.905     0.910       168
  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1